<a href="https://colab.research.google.com/github/jcdevaney/aveMaria/blob/main/aveMaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries and data

In [ ]:
import numpy as np
import pandas as pd

#import csv files
!git clone https://github.com/jcdevaney/aveMaria.git

fatal: destination path 'aveMaria' already exists and is not an empty directory.


## Function to sort data into conditions for the regression analysis

In [ ]:
def getConditionVals(avm, conditions):

    res = {}

    for i in range(6):

        res = {}

        # all semitones
        res['allSTupACAP'] = avm[0:3, conditions['STup']]
        res['allSTupACCOM'] = avm[3:6, conditions['STup']]
        res['allSTdnACAP'] = avm[0:3, conditions['STdn']]
        res['allSTdnACCOM'] = avm[3:6, conditions['STdn']]
 
        # semitone conditions
        res['ltACAP'] = avm[0:3, conditions['LT']]
        res['ltACCOM'] = avm[3:6, conditions['LT']]
        res['ABbupACAP'] = avm[0:3, conditions['ABbup']]
        res['ABbupACCOM'] = avm[3:6, conditions['ABbup']]
        res['ABbdnACAP'] = avm[0:3, conditions['ABbdn']]
        res['ABbdnACCOM'] = avm[3:6, conditions['ABbdn']]
        res['OSTupACAP'] = avm[0:3, conditions['OSTup']]
        res['OSTupACCOM'] = avm[3:6, conditions['OSTup']]
        res['OSTdnACAP'] = avm[0:3, conditions['OSTdn']]
        res['OSTdnACCOM'] = avm[3:6, conditions['OSTdn']]
        res['lsACAP'] = avm[0:3, conditions['STls']]
        res['lsACCOM'] = avm[3:6, conditions['STls']]
        res['slACAP'] = avm[0:3, conditions['STsl']]
        res['slACCOM'] = avm[3:6, conditions['STsl']]
        res['eqACAP'] = avm[0:3, conditions['STeq']]
        res['eqACCOM'] = avm[3:6, conditions['STeq']]
        
      #  accomConds = ['ACAP', 'ACCOM']
      #  accomIdx = [range(3), range(3, 6)]
      #  intCondsST = ['LT', 'ABbup', 'ABbdn', 'OSTup', 'OSTdn']
      #  durCondsST = ['STls', 'STsl', 'STeq']
        
       # n = 1
       # res['st']['combCond'] = []
        
       # for k in range(len(intCondsST)):
       #     for m in range(len(durCondsST)):
       #         for j in range(len(accomConds)):
       #             if set(conditions[durCondsST[m]]).intersection(conditions[intCondsST[k]]):
       #                 res['st'][intCondsST[k] + durCondsST[m] + accomConds[j]] = avm[accomIdx[j], list(set(conditions[durCondsST[m]]).intersection(conditions[intCondsST[k]]))]
       #                 res['st']['combCond'].append(intCondsST[k] + accomConds[j] + durCondsST[m])
       #                 n += 1
    
    return res


# Read CSV files

In [ ]:
# open csv files

npCSV = list()
pCSV = list()

for i in range(5):
  csv_path1 = "aveMaria/avmNP%d.csv" % (i+1)
  npCSV = pd.read_csv(csv_path1, header = None)
  csv_path2 = "aveMaria/avmP%d.csv" % (i+1)
  pCSV = pd.read_csv(csv_path2, header = None)

Set up conditions for regression

In [ ]:
# Semitone interval conditions
conditions = {}
conditions['LT'] = np.array([10, 27])
conditions['ABbup'] = np.array([2, 36, 65, 72])
conditions['ABbdn'] = np.array([1, 7, 64, 71])
conditions['OSTup'] = np.array([25, 42, 50])
conditions['OSTdn'] = np.array([19, 22, 41, 49, 52, 67])
conditions['STup'] = np.sort(np.concatenate([conditions['LT'], conditions['ABbup'], conditions['OSTup']]))
conditions['STdn'] = np.sort(np.concatenate([conditions['ABbdn'], conditions['OSTdn']]))
conditions['STall'] = np.sort(np.concatenate([conditions['STup'], conditions['STdn']]))
conditions['STls'] = np.array([1,19,42,71])
conditions['STsl'] = np.array([2,10,27,41])
conditions['STeq'] = np.array([7,22,25,36,49,50,52,64,65,67,72])

conIntsNP=getConditionVals(npCSV.to_numpy(), conditions);
conIntsP=getConditionVals(pCSV.to_numpy(), conditions);

## Run linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
# fit(X, y[, sample_weight]) - Fit linear model.
# get_params([deep]) - Get parameters for this estimator.
# predict(X) - Predict using the linear model.
# score(X, y[, sample_weight]) - Return the coefficient of determination of the prediction.
# set_params(**params) - Set the parameters of this estimator.

# run regression from vector and matrix in csv files
import matplotlib.pyplot as plt

csv_path3 = "aveMaria/AVMregressMat.csv" 
X = pd.read_csv(csv_path3, header = None)

csv_path4 = "aveMaria/AVMregressVec.csv" 
y = pd.read_csv(csv_path4, header = None)

reg = LinearRegression().fit(X, y)
print("R2 = " + str(reg.score(X, y)))
print("Regression coefficients = " + str(reg.coef_))
print(reg.intercept_)


R2 = 0.2831194551306353
Regression coefficients = [[ -7.50696541   3.9873272   13.86190856  20.90297862   7.69337314
   -6.75131511   1.88603001 -14.11853515  -9.54105332   9.61832168
   -7.08439814   0.        ]]
[182.52606731]


In [ ]:
import statsmodels.api as sm
alpha = 0.05 # 95% confidence interval
lr = sm.OLS(y, X).fit()
#print(lr.rsquared)
#print(lr.rsquared_adj)
#print(lr.pvalues)
print(lr.f_test)
print(lr.conf_int(alpha))

<bound method LikelihoodModelResults.f_test of <statsmodels.regression.linear_model.OLSResults object at 0x7fef87a15850>>
             0           1
0   -18.422164    3.408233
1    -1.292255    9.266909
2     6.829142   20.894675
3    10.152374   31.653583
4     1.074407   14.312340
5   -12.263340   -1.239290
6    -4.071822    7.843882
7   -20.108029   -8.129041
8   -15.498906   -3.583201
9     3.660469   15.576174
10  -12.892325   -1.276471
11  172.634390  192.417745
